# DDPG

In this Jupyter notebook I will try to show the differences between a normal critic actor algorithm and DDPG and how we can implement DDPG in stable baselines.

## The Differences in algorithms:

### Actor Cirtic
```python
def mlp(x, hidden_sizes=(32,), activation=tf.tanh, output_activation=None):
    for h in hidden_sizes[:-1]:
        x = tf.layers.dense(x, units=h, activation=activation)
    return tf.layers.dense(x, units=hidden_sizes[-1], activation=output_activation)

def mlp_gaussian_policy(x, a, hidden_sizes, activation, output_activation, action_space):
    act_dim = a.shape.as_list()[-1]
    mu = mlp(x, list(hidden_sizes)+[act_dim], activation, output_activation)
    log_std = tf.get_variable(name='log_std', initializer=-0.5*np.ones(act_dim, dtype=np.float32))
    std = tf.exp(log_std)
    pi = mu + tf.random_normal(tf.shape(mu)) * std
    logp = gaussian_likelihood(a, mu, log_std)
    logp_pi = gaussian_likelihood(pi, mu, log_std)
    return pi, logp, logp_pi

def mlp_actor_critic(x, a, hidden_sizes=(64,64), activation=tf.tanh, 
                     output_activation=None, policy=None, action_space=None):
    # Actor
    policy = mlp_gaussian_policy
    with tf.variable_scope('pi'):
        pi, logp, logp_pi = policy(x, a, hidden_sizes, activation, output_activation, action_space)
    # Critic
    with tf.variable_scope('v'):
        v = tf.squeeze(mlp(x, list(hidden_sizes)+[1], activation, None), axis=1)
    return pi, logp, logp_pi, v
```

### DDPG
```python
def mlp(x, hidden_sizes=(32,), activation=tf.tanh, output_activation=None):
    for h in hidden_sizes[:-1]:
        x = tf.layers.dense(x, units=h, activation=activation)
    return tf.layers.dense(x, units=hidden_sizes[-1], activation=output_activation)

def mlp_actor_critic(x, a, hidden_sizes=(400,300), activation=tf.nn.relu, 
                     output_activation=tf.tanh, action_space=None):
    act_dim = a.shape.as_list()[-1]
    act_limit = action_space.high[0]
    # Actor
    with tf.variable_scope('pi'):
        pi = act_limit * mlp(x, list(hidden_sizes)+[act_dim], activation, output_activation)
    # Critic (inputs used action and state)
    with tf.variable_scope('q'):
        q = tf.squeeze(mlp(tf.concat([x,a], axis=-1), list(hidden_sizes)+[1], activation, None), axis=1)
    # Critic (inputs action from the policy and state)
    with tf.variable_scope('q', reuse=True):
        q_pi = tf.squeeze(mlp(tf.concat([x,pi], axis=-1), list(hidden_sizes)+[1], activation, None), axis=1)
    return pi, q, q_pi
```

## The Differences in Lose function:

### Actor Critic objectives:
```python
# VPG objectives:
# Use Policy gradinet method to train the policy
pi_loss = -tf.reduce_mean(logp * adv_ph)
# Train the critic: The critic should 
# give the summ of reward as output
v_loss = tf.reduce_mean((ret_ph - v)**2)
```

### DDPG objectives:
```python
## DDPG losses
# Train the policy
pi_loss = -tf.reduce_mean(q_pi)

# Train the critic based on the used action in the trajectory 
# Bellman backup for Q function,  use the policy to calculate the target for the q function
backup = tf.stop_gradient(r_ph + gamma*(1-d_ph)*q_pi_targ)
q_loss = tf.reduce_mean((q-backup)**2)
```

## DDPG Example with stable_baselines

In [57]:
import gym
import numpy as np

from stable_baselines.ddpg.policies import MlpPolicy
from stable_baselines.ddpg.policies import FeedForwardPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.ddpg.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
from stable_baselines import DDPG

Reward is 100 for reaching the target of the hill on the right hand side, minus the squared sum of actions from start to goal.

This reward function raises an exploration challenge, because if the agent does not reach the target soon enough, it will figure out that it is better not to move, and won't find the target anymore.

Note that this reward is unusual with respect to most published work, where the goal was to reach the target as fast as possible, hence favouring a bang-bang strategy.

In [58]:
env = DummyVecEnv([lambda: gym.make('MountainCarContinuous-v0')])
# Environment for evaluation
eval_env = gym.make('MountainCarContinuous-v0')

### Exploration
DDPG trains a deterministic policy in an off-policy way. Because the policy is deterministic, if the agent were to explore on-policy, in the beginning it would probably not try a wide enough variety of actions to find useful learning signals. To make DDPG policies explore better, we add noise to their actions at training time. The authors of the original DDPG paper recommended time-correlated OU noise. To facilitate getting higher-quality training data, you may reduce the scale of the noise over the course of training.

In [59]:
# the noise objects for DDPG
n_actions = env.action_space.shape[-1]
param_noise = None
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=float(0.5) * np.ones(n_actions))

### Create Custom Policy 

In [60]:
# Custom MLP policy of two layers of size 16 each
class CustomPolicy(FeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomPolicy, self).__init__(*args, **kwargs,
                                           layers=[32, 32,32],
                                           layer_norm=False,
                                           feature_extraction="mlp")

### Define a  Model:
Some parameters that we have to define:
* **policy** – (DDPGPolicy or str) The policy model to use (MlpPolicy, CnnPolicy, LnMlpPolicy, …)
* **env** – (Gym environment or str) The environment to learn from (if registered in Gym, can be str)
* **gamma** – (float) the discount factor
* **eval_env** – (Gym Environment) the evaluation environment (can be None)
* **nb_train_steps** – (int) the number of training steps (how many times we sample batches from Replay Buffer)
* **nb_rollout_steps** – (int) the number of rollout(epsiode) steps
* **nb_eval_steps** – (int) the number of evalutation steps
* **param_noise** – (AdaptiveParamNoiseSpec) the parameter noise type (can be None)
* **action_noise** – (ActionNoise) the action noise type (can be None)
* **param_noise_adaption_interval** – (int) apply param noise every N steps
* **tau** – (float) the soft update coefficient (keep old values, between 0 and 1)
* **normalize_returns** – (bool) should the critic output be normalized
* **normalize_observations** – (bool) should the observation be normalized
* **batch_size** – (int) the size of the batch for learning the policy
* **observation_range** – (tuple) the bounding values for the observation
* **return_range** – (tuple) the bounding values for the critic output
* **critic_l2_reg** – (float) l2 regularizer coefficient
* **actor_lr** – (float) the actor learning rate
* **critic_lr** – (float) the critic learning rate
* **clip_norm** – (float) clip the gradients (disabled if None)
* **reward_scale** – (float) the value the reward should be scaled by
* **render** – (bool) enable rendering of the environment
* **render_eval** – (bool) enable rendering of the evalution environment
* **buffer_size** – (int) the max number of transitions to store, size of the replay buffer
* **random_exploration** – (float) Probability of taking a random action (as in an epsilon-greedy strategy) This is not needed for DDPG normally but can help exploring when using HER + DDPG. This hack was present in the original OpenAI Baselines repo (DDPG + HER)
* **verbose** – (int) the verbosity level: 0 none, 1 training information, 2 tensorflow debug
* **tensorboard_log** – (str) the log location for tensorboard (if None, no logging)
* **policy_kwargs** – (dict) additional arguments to be passed to the policy on creation
* **full_tensorboard_log** – (bool) enable additional logging when using tensorboard WARNING: this logging can take a lot of space quickly


In [66]:
model = DDPG(policy = MlpPolicy, 
             env = env,
             render = False,
             nb_train_steps = 50,
             nb_rollout_steps = 200,
             eval_env = eval_env,
             render_eval = False,
             nb_eval_steps = 2,
             gamma = 0.999,
             buffer_size = 5000, 
             param_noise=param_noise, 
             action_noise=action_noise,
             verbose=1,
             tensorboard_log = '/home/karam/workspaces/reinforcement_learning/RL_AV'
            )

#### Train the Model
* **total_timesteps** – (int) The total number of samples to train on
* **callback** – (function (dict, dict)) -> boolean function called at every steps with state of the algorithm. It takes the local and global variables. If it returns False, training is aborted.
* **seed** – (int) The initial seed for training, if None: keep current seed
* **log_interval** – (int) The number of timesteps before logging.
* **tb_log_name** – (str) the name of the run for tensorboard log
reset_num_timesteps – (bool) whether or not to reset the current timestep number (used in logging)

In [67]:
model.learn(total_timesteps=300000,  log_interval=4)

---------------------------------------
| eval/Q                  | -0.00106  |
| eval/episodes           | 0         |
| eval/return             | nan       |
| eval/return_history     | nan       |
| reference_Q_mean        | -0.0421   |
| reference_Q_std         | 0.0346    |
| reference_action_mean   | -0.00825  |
| reference_action_std    | 0.0017    |
| reference_actor_Q_mean  | 0.000233  |
| reference_actor_Q_std   | 0.00293   |
| rollout/Q_mean          | 0.000252  |
| rollout/actions_mean    | 0.38      |
| rollout/actions_std     | 0.496     |
| rollout/episode_steps   | nan       |
| rollout/episodes        | 0         |
| rollout/return          | nan       |
| rollout/return_history  | nan       |
| total/duration          | 1.33      |
| total/episodes          | 0         |
| total/epochs            | 1         |
| total/steps             | 796       |
| total/steps_per_second  | 598       |
| train/loss_actor        | -0.000442 |
| train/loss_critic       | 3.04e-06  |



--------------------------------------
| eval/Q                  | 0.00243  |
| eval/episodes           | 0        |
| eval/return             | nan      |
| eval/return_history     | nan      |
| reference_Q_mean        | -0.0385  |
| reference_Q_std         | 0.0346   |
| reference_action_mean   | -0.00472 |
| reference_action_std    | 0.0153   |
| reference_actor_Q_mean  | 0.00265  |
| reference_actor_Q_std   | 0.000819 |
| rollout/Q_mean          | 0.00103  |
| rollout/actions_mean    | 0.271    |
| rollout/actions_std     | 0.543    |
| rollout/episode_steps   | 999      |
| rollout/episodes        | 7        |
| rollout/return          | -37.4    |
| rollout/return_history  | -37.4    |
| total/duration          | 10.7     |
| total/episodes          | 7        |
| total/epochs            | 1        |
| total/steps             | 7196     |
| total/steps_per_second  | 671      |
| train/loss_actor        | -0.00241 |
| train/loss_critic       | 3.92e-08 |
| train/param_noise_di..


--------------------------------------
| eval/Q                  | 0.00284  |
| eval/episodes           | 0        |
| eval/return             | nan      |
| eval/return_history     | nan      |
| reference_Q_mean        | -0.0382  |
| reference_Q_std         | 0.0347   |
| reference_action_mean   | 0.00574  |
| reference_action_std    | 0.0134   |
| reference_actor_Q_mean  | 0.00298  |
| reference_actor_Q_std   | 0.000762 |
| rollout/Q_mean          | 0.00153  |
| rollout/actions_mean    | 0.107    |
| rollout/actions_std     | 0.612    |
| rollout/episode_steps   | 999      |
| rollout/episodes        | 13       |
| rollout/return          | -39.2    |
| rollout/return_history  | -39.2    |
| total/duration          | 20.3     |
| total/episodes          | 13       |
| total/epochs            | 1        |
| total/steps             | 13596    |
| total/steps_per_second  | 670      |
| train/loss_actor        | -0.00218 |
| train/loss_critic       | 2.14e-08 |
| train/param_noise_di..


---------------------------------------
| eval/Q                  | -0.101    |
| eval/episodes           | 0         |
| eval/return             | nan       |
| eval/return_history     | nan       |
| reference_Q_mean        | -0.0487   |
| reference_Q_std         | 0.117     |
| reference_action_mean   | -0.0395   |
| reference_action_std    | 0.0161    |
| reference_actor_Q_mean  | -0.0206   |
| reference_actor_Q_std   | 0.12      |
| rollout/Q_mean          | -0.000232 |
| rollout/actions_mean    | 0.0522    |
| rollout/actions_std     | 0.59      |
| rollout/episode_steps   | 961       |
| rollout/episodes        | 20        |
| rollout/return          | -19.4     |
| rollout/return_history  | -19.4     |
| total/duration          | 29.9      |
| total/episodes          | 20        |
| total/epochs            | 1         |
| total/steps             | 19996     |
| total/steps_per_second  | 670       |
| train/loss_actor        | 0.101     |
| train/loss_critic       | 3.73      |


--------------------------------------
| eval/Q                  | -0.179   |
| eval/episodes           | 0        |
| eval/return             | nan      |
| eval/return_history     | nan      |
| reference_Q_mean        | -0.141   |
| reference_Q_std         | 0.0992   |
| reference_action_mean   | 0.0338   |
| reference_action_std    | 0.00525  |
| reference_actor_Q_mean  | -0.108   |
| reference_actor_Q_std   | 0.107    |
| rollout/Q_mean          | -0.0378  |
| rollout/actions_mean    | 0.0532   |
| rollout/actions_std     | 0.612    |
| rollout/episode_steps   | 965      |
| rollout/episodes        | 27       |
| rollout/return          | -21.9    |
| rollout/return_history  | -21.9    |
| total/duration          | 40.2     |
| total/episodes          | 27       |
| total/epochs            | 1        |
| total/steps             | 26396    |
| total/steps_per_second  | 657      |
| train/loss_actor        | 0.153    |
| train/loss_critic       | 0.997    |
| train/param_noise_di..

...


--------------------------------------
| eval/Q                  | 103      |
| eval/episodes           | 2        |
| eval/return             | 99.9     |
| eval/return_history     | 99.9     |
| reference_Q_mean        | 1.74     |
| reference_Q_std         | 4.56     |
| reference_action_mean   | 0.679    |
| reference_action_std    | 0.169    |
| reference_actor_Q_mean  | 1.93     |
| reference_actor_Q_std   | 4.64     |
| rollout/Q_mean          | 5.4      |
| rollout/actions_mean    | 0.15     |
| rollout/actions_std     | 0.65     |
| rollout/episode_steps   | 544      |
| rollout/episodes        | 495      |
| rollout/return          | 42.1     |
| rollout/return_history  | 77.9     |
| total/duration          | 464      |
| total/episodes          | 495      |
| total/epochs            | 1        |
| total/steps             | 269596   |
| total/steps_per_second  | 581      |
| train/loss_actor        | -26.1    |
| train/loss_critic       | 27.8     |
| train/param_noise_di..


--------------------------------------
| eval/Q                  | 103      |
| eval/episodes           | 2        |
| eval/return             | 99.9     |
| eval/return_history     | 99.9     |
| reference_Q_mean        | 1.69     |
| reference_Q_std         | 5.19     |
| reference_action_mean   | 0.558    |
| reference_action_std    | 0.21     |
| reference_actor_Q_mean  | 1.94     |
| reference_actor_Q_std   | 5.3      |
| rollout/Q_mean          | 5.63     |
| rollout/actions_mean    | 0.161    |
| rollout/actions_std     | 0.651    |
| rollout/episode_steps   | 546      |
| rollout/episodes        | 505      |
| rollout/return          | 42       |
| rollout/return_history  | 73.1     |
| total/duration          | 476      |
| total/episodes          | 505      |
| total/epochs            | 1        |
| total/steps             | 275996   |
| total/steps_per_second  | 580      |
| train/loss_actor        | -17.9    |
| train/loss_critic       | 9.52     |
| train/param_noise_di..


--------------------------------------
| eval/Q                  | 108      |
| eval/episodes           | 2        |
| eval/return             | 99.9     |
| eval/return_history     | 99.9     |
| reference_Q_mean        | 1.88     |
| reference_Q_std         | 5.21     |
| reference_action_mean   | 0.664    |
| reference_action_std    | 0.188    |
| reference_actor_Q_mean  | 2.1      |
| reference_actor_Q_std   | 5.47     |
| rollout/Q_mean          | 5.71     |
| rollout/actions_mean    | 0.174    |
| rollout/actions_std     | 0.653    |
| rollout/episode_steps   | 551      |
| rollout/episodes        | 512      |
| rollout/return          | 40.8     |
| rollout/return_history  | 63.5     |
| total/duration          | 488      |
| total/episodes          | 512      |
| total/epochs            | 1        |
| total/steps             | 282396   |
| total/steps_per_second  | 579      |
| train/loss_actor        | -3.06    |
| train/loss_critic       | 0.132    |
| train/param_noise_di..


--------------------------------------
| eval/Q                  | 103      |
| eval/episodes           | 2        |
| eval/return             | 99.9     |
| eval/return_history     | 99.9     |
| reference_Q_mean        | 1.72     |
| reference_Q_std         | 5.84     |
| reference_action_mean   | 0.0304   |
| reference_action_std    | 0.653    |
| reference_actor_Q_mean  | 2.02     |
| reference_actor_Q_std   | 5.93     |
| rollout/Q_mean          | 6.4      |
| rollout/actions_mean    | 0.177    |
| rollout/actions_std     | 0.655    |
| rollout/episode_steps   | 537      |
| rollout/episodes        | 538      |
| rollout/return          | 42.6     |
| rollout/return_history  | 61.7     |
| total/duration          | 500      |
| total/episodes          | 538      |
| total/epochs            | 1        |
| total/steps             | 288796   |
| total/steps_per_second  | 578      |
| train/loss_actor        | -47.1    |
| train/loss_critic       | 10.8     |
| train/param_noise_di..


--------------------------------------
| eval/Q                  | 99.7     |
| eval/episodes           | 2        |
| eval/return             | 99.9     |
| eval/return_history     | 99.9     |
| reference_Q_mean        | 2.14     |
| reference_Q_std         | 6.11     |
| reference_action_mean   | -0.349   |
| reference_action_std    | 0.77     |
| reference_actor_Q_mean  | 3        |
| reference_actor_Q_std   | 6.25     |
| rollout/Q_mean          | 7.44     |
| rollout/actions_mean    | 0.18     |
| rollout/actions_std     | 0.658    |
| rollout/episode_steps   | 505      |
| rollout/episodes        | 584      |
| rollout/return          | 46.2     |
| rollout/return_history  | 64.7     |
| total/duration          | 511      |
| total/episodes          | 584      |
| total/epochs            | 1        |
| total/steps             | 295196   |
| total/steps_per_second  | 577      |
| train/loss_actor        | -56.8    |
| train/loss_critic       | 21       |
| train/param_noise_di..

Save the model

In [54]:
model.save("ddpg_mountain")

Test your model

In [69]:
obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

env.close()